## 파일 목적
Surrogate Model(single-objective) 구현 및  
Surrogate Model(LightGBM, CatBoost, TabPFN)으로 multi-objective 구현하기 위해, end-to-end로 잘 동작하는 지 확인 

## Surrogate Model(LightGBM)으로 multi-objective 구현

In [1]:
import src.datasets as datasets
import src.surrogate as surrogate
import src.search as search

import numpy as np
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd
df = pd.read_csv('./data/concrete_processed.csv')
df

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,0.089726,0.619702,0.000000,0.757426,0.000000,0.496512,0.422059,0.000000,0.357597
1,0.152740,0.123356,0.621189,0.309901,0.488688,0.813372,0.562353,-0.666667,0.274815
2,0.337900,0.000000,0.478261,0.598020,0.248869,0.453198,0.753529,0.000000,0.348904
3,0.374429,0.333236,0.000000,1.000000,0.000000,0.380814,0.191176,0.000000,0.564681
4,0.120548,0.536101,0.000000,0.656436,0.411765,0.716279,0.269706,0.000000,0.207084
...,...,...,...,...,...,...,...,...,...
921,0.089726,0.619702,0.000000,0.757426,0.000000,0.496512,0.422059,-1.000000,0.104580
922,0.445662,0.000000,0.587206,0.473267,0.429864,0.644767,0.436765,-1.190476,0.254055
923,0.075342,0.000000,0.829585,0.524752,0.452489,0.465116,0.588235,0.000000,0.142208
924,0.398174,0.339082,0.451274,0.520792,0.402715,0.200872,0.480294,0.000000,0.544310


In [4]:
target_cols = ['strength', 'cement', 'water']

In [5]:
X = df.drop(columns=target_cols)
y = df[target_cols]    

In [6]:
X

,slag,ash,superplastic,coarseagg,fineagg,age
0,0.619702,0.000000,0.000000,0.496512,0.422059,0.000000
1,0.123356,0.621189,0.488688,0.813372,0.562353,-0.666667
2,0.000000,0.478261,0.248869,0.453198,0.753529,0.000000
3,0.333236,0.000000,0.000000,0.380814,0.191176,0.000000
4,0.536101,0.000000,0.411765,0.716279,0.269706,0.000000
...,...,...,...,...,...,...
921,0.619702,0.000000,0.000000,0.496512,0.422059,-1.000000
922,0.000000,0.587206,0.429864,0.644767,0.436765,-1.190476
923,0.000000,0.829585,0.452489,0.465116,0.588235,0.000000
924,0.339082,0.451274,0.402715,0.200872,0.480294,0.000000


In [7]:
y

,strength,cement,water
0,0.357597,0.089726,0.757426
1,0.274815,0.152740,0.309901
2,0.348904,0.337900,0.598020
3,0.564681,0.374429,1.000000
4,0.207084,0.120548,0.656436
...,...,...,...
921,0.104580,0.089726,0.757426
922,0.254055,0.445662,0.473267
923,0.142208,0.075342,0.524752
924,0.544310,0.398174,0.520792


In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    shuffle=True

    
)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape: ", X_test.shape)
print("y_test shape: ", y_test.shape)

X_train shape: (740, 6)
y_train shape: (740, 3)
X_test shape:  (186, 6)
y_test shape:  (186, 3)


In [9]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
print(X_train.shape, type(X_train))
print(y_train.shape, type(y_train))
print(X_test.shape, type(X_test))
print(y_test.shape, type(y_test))

(740, 6) <class 'numpy.ndarray'>
(740, 3) <class 'numpy.ndarray'>
(186, 6) <class 'numpy.ndarray'>
(186, 3) <class 'numpy.ndarray'>


In [10]:
def lightgbm_multi_train(X_train: np.ndarray,
                         y_train: np.ndarray, 
                         params: dict = None):
    
    if params is None:
        params = {
            "objective": "regression",   
            "boosting_type": "gbdt",     
            "learning_rate": 0.05,       
            "num_leaves": 31,
            "max_depth": -1,
            "subsample": 0.8,
            "colsample_bytree": 0.8,
            "n_jobs": -1,
            "random_state": 42
        }
    # X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, shuffle=True)
    base_estimator = LGBMRegressor(**params, n_estimators=1000)
    multi_model = MultiOutputRegressor(base_estimator)
    multi_model.fit(X_train, y_train,
                    # eval_set=[(X_val, y_val)],
                    # eval_metric="rmse",
                    # callbacks=[
                    # early_stopping(stopping_rounds=50),
                    # log_evaluation(period=100)
                    # ]
                    )

    return multi_model

In [ ]:
model = lightgbm_multi_train(X_train, y_train)

In [12]:
y_pred = model.predict(X_test)
y_pred.shape

(186, 3)

In [62]:
y_train.shape

(740, 3)

In [23]:
# 모델 저장
import joblib

def lightgbm_save(model, path):
    """MultiOutputRegressor의 개별 LightGBM 모델을 저장"""
    # models = model.estimators_  # 내부의 개별 모델 리스트 가져오기
    # joblib.dump(models, path)
    joblib.dump(model, path)

In [27]:
lightgbm_save(model, './model_save/multi_lightgbm.pkl')

In [30]:
def lightgbm_load(path):
    """joblib을 이용해 MultiOutputRegressor를 불러오기"""
    return joblib.load(path)

In [31]:
loaded_model = lightgbm_load('./model_save/multi_lightgbm.pkl')

In [34]:
y_pred = loaded_model.predict(X_test)
y_pred.shape

(186, 3)

In [96]:
import numpy as np

def lightgbm_multi_evaluate(model, y_train, y_pred, y_test, target_cols):
    rmse_list, mae_list, r2_list = [], [], []
    
    for idx, col in enumerate(target_cols):
        y_true_i = y_test[:, idx]  # target_dict의 key를 인덱스로 변환
        y_pred_i = y_pred[:, idx]
        
        mae_i = np.mean(np.abs(y_true_i - y_pred_i))
        mse_i = np.mean((y_true_i - y_pred_i) ** 2)
        rmse_i = np.sqrt(mse_i)
        
        sse_i = np.sum((y_true_i - y_pred_i) ** 2)
        sst_i = np.sum(y_true_i - np.mean(y_train) ** 2)
        
        r2_i = 1 - sse_i / sst_i
        
        rmse_list.append(rmse_i)
        mae_list.append(mae_i)
        r2_list.append(r2_i)
        
        print(f"Target '{col}' - RMSE: {rmse_i:.4f}, MAE: {mae_i:.4f}, R2: {r2_i:.4f}")
    
    rmse_mean = np.mean(rmse_list)
    mae_mean = np.mean(mae_list)
    r2_mean = np.mean(r2_list)
    
    print(f"[Average Metrics] RMSE: {rmse_mean:.4f}, MAE: {mae_mean:.4f}, R2: {r2_mean:.4f}")
    return rmse_mean, mae_mean, r2_mean

In [97]:
rmse_mean, mae_mean, r2_mean = lightgbm_multi_evaluate(model,
                        y_train, 
                        y_pred,
                        y_test,
                        target_cols
                        )

print(f'lightGBM 모델 mulit-objective RMSE: {rmse_mean:.4f}')
print(f'lightGBM 모델 mulit-objective MAE: {mae_mean:.4f}')
print(f'lightGBM 모델 mulit-objective R^2: {r2_mean:.4f}')

Target 'strength' - RMSE: 0.0803, MAE: 0.0450, R2: 0.9674
Target 'cement' - RMSE: 0.0666, MAE: 0.0248, R2: 0.9755
Target 'water' - RMSE: 0.0545, MAE: 0.0207, R2: 0.9911
[Average Metrics] RMSE: 0.0672, MAE: 0.0302, R2: 0.9780
lightGBM 모델 mulit-objective RMSE: 0.0672
lightGBM 모델 mulit-objective MAE: 0.0302
lightGBM 모델 mulit-objective R^2: 0.9780


## Surrogate Model(CatBoost)으로 single-objective 구현

In [35]:
import src.datasets as datasets
import src.surrogate as surrogate
import src.search as search

In [36]:
load_data_func = getattr(datasets, f'cement_data')
X_train, X_test, y_train, y_test = load_data_func('./data/concrete_processed.csv')

In [37]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(740, 8)
(740, 1)
(186, 8)
(186, 1)


In [38]:
# 비교를 위한 선형회귀
from sklearn.linear_model import LinearRegression
baseline_model = LinearRegression()
baseline_model.fit(X_train, y_train)
baseline_pred = baseline_model.predict(X_test)
baseline_r2 = 1 - np.sum((y_test - baseline_pred) ** 2) / (np.sum((y_test - y_train.mean()) ** 2) + 1e-10)
print(f'선형회귀 모델 R^2: {baseline_r2.item() :.4f}')

선형회귀 모델 R^2: 0.7556


In [39]:
from catboost import CatBoostRegressor

In [40]:
model = CatBoostRegressor(
    iterations=2000,        # 학습 반복 횟수
    depth=7,                # 트리 깊이
    learning_rate=0.05,     # 학습률
    bagging_temperature=1, # 앙상블 다양성을 조절 (1~3 추천)
    # l2_leaf_reg=5,         # L2 정규화 (3~10 사이에서 튜닝 가능)
    loss_function='RMSE',   # 손실 함수 (회귀 문제이므로 RMSE 사용)
    # eval_metric='RMSE',     # 평가 지표
    random_seed=42,
    verbose=100,            # 학습 과정 출력
    early_stopping_rounds=100  # 조기 종료
)

model.fit(X_train, y_train)

0:	learn: 0.2125008	total: 48.1ms	remaining: 1m 36s
100:	learn: 0.0540607	total: 142ms	remaining: 2.68s
200:	learn: 0.0423818	total: 232ms	remaining: 2.08s


300:	learn: 0.0352422	total: 322ms	remaining: 1.82s
400:	learn: 0.0305735	total: 411ms	remaining: 1.64s
500:	learn: 0.0272640	total: 501ms	remaining: 1.5s
600:	learn: 0.0249423	total: 589ms	remaining: 1.37s
700:	learn: 0.0231242	total: 680ms	remaining: 1.26s
800:	learn: 0.0218541	total: 790ms	remaining: 1.18s
900:	learn: 0.0206984	total: 882ms	remaining: 1.08s
1000:	learn: 0.0198208	total: 985ms	remaining: 983ms
1100:	learn: 0.0191964	total: 1.1s	remaining: 897ms
1200:	learn: 0.0186435	total: 1.21s	remaining: 802ms
1300:	learn: 0.0181889	total: 1.29s	remaining: 696ms
1400:	learn: 0.0177914	total: 1.38s	remaining: 590ms
1500:	learn: 0.0174438	total: 1.48s	remaining: 492ms
1600:	learn: 0.0171564	total: 1.57s	remaining: 392ms
1700:	learn: 0.0169187	total: 1.66s	remaining: 292ms
1800:	learn: 0.0167320	total: 1.76s	remaining: 194ms
1900:	learn: 0.0165502	total: 1.86s	remaining: 96.8ms
1999:	learn: 0.0164236	total: 1.95s	remaining: 0us


In [41]:
def eval_surrogate_model(y_train,y_pred, y_test):
    rmse = np.sqrt(np.mean((y_test - y_pred) ** 2))
    mae = np.mean(np.abs(y_test - y_pred))
    SSE = np.sum(np.square(y_test - y_pred))    
    SST = np.sum(np.square(y_test - y_train.mean()))
    r2 = 1 - SSE/SST
    return rmse, mae, r2

In [42]:
y_pred = model.predict(X_test)
print(y_pred.shape)
if y_pred.ndim == 1:
        y_pred = y_pred.reshape(-1, 1)
print(y_pred.shape)

(186,)
(186, 1)


In [43]:
rmse, mae, r2 = eval_surrogate_model(y_train, y_pred, y_test)
print(f'catboost 모델 RMSE: {rmse:.4f}')
print(f'catboost 모델 MAE: {mae:.4f}')
print(f'catboost 모델 R^2: {r2:.4f}')

catboost 모델 RMSE: 0.0496
catboost 모델 MAE: 0.0302
catboost 모델 R^2: 0.9430


In [44]:
# 모델저장
def catboost_save(model, path):
    """CatBoost 모델 저장"""
    model.save_model(path, format="cbm")  # CatBoost 전용 포맷으로 저장

In [45]:
catboost_save(model,'./model_save/catboost_model.cbm')

In [46]:
# 모델불러오기
def catboost_load(path):
    """CatBoost 모델 불러오기"""
    model = CatBoostRegressor()  # 회귀 모델이면 CatBoostRegressor, 분류 모델이면 CatBoostClassifier
    model.load_model(path)
    return model

In [48]:
loaded_model = catboost_load('./model_save/catboost_model.cbm')

In [49]:
y_pred = loaded_model.predict(X_test)

In [50]:
y_pred.shape

(186,)

## Surrogate Model(CatBoost)으로 multi-objective 구현

In [51]:
import pandas as pd
df = pd.read_csv('./data/concrete_processed.csv')
df

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,0.089726,0.619702,0.000000,0.757426,0.000000,0.496512,0.422059,0.000000,0.357597
1,0.152740,0.123356,0.621189,0.309901,0.488688,0.813372,0.562353,-0.666667,0.274815
2,0.337900,0.000000,0.478261,0.598020,0.248869,0.453198,0.753529,0.000000,0.348904
3,0.374429,0.333236,0.000000,1.000000,0.000000,0.380814,0.191176,0.000000,0.564681
4,0.120548,0.536101,0.000000,0.656436,0.411765,0.716279,0.269706,0.000000,0.207084
...,...,...,...,...,...,...,...,...,...
921,0.089726,0.619702,0.000000,0.757426,0.000000,0.496512,0.422059,-1.000000,0.104580
922,0.445662,0.000000,0.587206,0.473267,0.429864,0.644767,0.436765,-1.190476,0.254055
923,0.075342,0.000000,0.829585,0.524752,0.452489,0.465116,0.588235,0.000000,0.142208
924,0.398174,0.339082,0.451274,0.520792,0.402715,0.200872,0.480294,0.000000,0.544310


In [52]:
target_cols = ['strength', 'cement', 'water']
X = df.drop(columns=target_cols)
y = df[target_cols]    

In [53]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

In [54]:
print(X_train.shape, type(X_train))
print(y_train.shape, type(y_train))
print(X_test.shape, type(X_test))
print(y_test.shape, type(y_test))

(740, 6) <class 'pandas.core.frame.DataFrame'>
(740, 3) <class 'pandas.core.frame.DataFrame'>
(186, 6) <class 'pandas.core.frame.DataFrame'>
(186, 3) <class 'pandas.core.frame.DataFrame'>


In [55]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
print(X_train.shape, type(X_train))
print(y_train.shape, type(y_train))
print(X_test.shape, type(X_test))
print(y_test.shape, type(y_test))

(740, 6) <class 'numpy.ndarray'>
(740, 3) <class 'numpy.ndarray'>
(186, 6) <class 'numpy.ndarray'>
(186, 3) <class 'numpy.ndarray'>


### 방법 1 : 개별모델학습해서 multi-objective 최적화

In [56]:
from catboost import CatBoostRegressor

In [ ]:
models = []
for i in range(y.shape[1]):
    model = CatBoostRegressor(
        iterations=2000,
        depth=7,
        learning_rate=0.05,
        loss_function='RMSE',
        random_seed=42,
        verbose=200
    )
    model.fit(X_train, y_train[:, i], early_stopping_rounds=100)
    models.append(model)

In [99]:
y_preds = []
for m in models:
    y_pred = m.predict(X_test)
    print(y_pred.shape)
    if y_pred.ndim == 1:
        y_pred = y_pred.reshape(-1, 1)
    print(y_pred.shape)
    y_preds.append(y_pred)
y_preds = np.column_stack(y_preds)
print(y_preds.shape)

# y_preds = np.column_stack([m.predict(X_test) for m in models])

(186,)
(186, 1)
(186,)
(186, 1)
(186,)
(186, 1)
(186, 3)


In [100]:
import numpy as np

def catboost_multi_evaluate(model, y_train, y_pred, y_test, target_cols):
    rmse_list, mae_list, r2_list = [], [], []
    
    for idx, col in enumerate(target_cols):
        y_true_i = y_test[:, idx]  # target_dict의 key를 인덱스로 변환
        y_pred_i = y_pred[:, idx]
        
        mae_i = np.mean(np.abs(y_true_i - y_pred_i))
        mse_i = np.mean((y_true_i - y_pred_i) ** 2)
        rmse_i = np.sqrt(mse_i)
        
        sse_i = np.sum((y_true_i - y_pred_i) ** 2)
        sst_i = np.sum(y_true_i - np.mean(y_train) ** 2)
        
        r2_i = 1 - sse_i / sst_i
        
        rmse_list.append(rmse_i)
        mae_list.append(mae_i)
        r2_list.append(r2_i)
        
        print(f"Target '{col}' - RMSE: {rmse_i:.4f}, MAE: {mae_i:.4f}, R2: {r2_i:.4f}")
    
    rmse_mean = np.mean(rmse_list)
    mae_mean = np.mean(mae_list)
    r2_mean = np.mean(r2_list)
    
    print(f"[Average Metrics] RMSE: {rmse_mean:.4f}, MAE: {mae_mean:.4f}, R2: {r2_mean:.4f}")
    return rmse_mean, mae_mean, r2_mean

In [101]:
rmse_mean, mae_mean, r2_mean = catboost_multi_evaluate(model,
                            y_train,    
                            y_preds,  
                            y_test,   
                            target_cols)
print(f'catboost 모델 mulit-objective RMSE: {rmse_mean:.4f}')
print(f'catboost 모델 mulit-objective MAE: {mae_mean:.4f}')
print(f'catboost 모델 mulit-objective R^2: {r2_mean:.4f}')

Target 'strength' - RMSE: 0.0796, MAE: 0.0421, R2: 0.9680
Target 'cement' - RMSE: 0.0641, MAE: 0.0224, R2: 0.9773
Target 'water' - RMSE: 0.0584, MAE: 0.0219, R2: 0.9898
[Average Metrics] RMSE: 0.0674, MAE: 0.0288, R2: 0.9784
catboost 모델 mulit-objective RMSE: 0.0674
catboost 모델 mulit-objective MAE: 0.0288
catboost 모델 mulit-objective R^2: 0.9784


### 방법 2 : custom loss function 활용 - 실패

In [224]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(740, 6)
(740, 3)
(186, 6)
(186, 3)


In [133]:
class MultiObjectiveLoss:
    def __init__(self, alpha=0.5, beta=0.5):
        """
        Multi-Objective 손실 함수 (RMSE + MAE)
        alpha: RMSE 가중치
        beta: MAE 가중치
        """
        self.alpha = alpha
        self.beta = beta
    
    def calc_ders_range(self, approxes, targets, weights):
        """
        approxes: 예측값 (logits)
        targets: 실제 값
        weights: 샘플 가중치 (필요 없을 경우 None)
        """
        assert len(approxes) == len(targets)
        
        derivatives = []
        second_derivatives = []
        
        for i in range(len(approxes)):
            error1 = approxes[i] - targets[i]  # RMSE 오차
            error2 = approxes[i] - targets[i]  # MAE 오차
            
            grad_rmse = error1 # RMSE 미분값
            grad_mae = np.sign(error2) # MAE 미분값
            
            grad = self.alpha * grad_rmse + self.beta * grad_mae  # Gradient (1차 미분)
            # Hessian (2차 미분) - 보통 1로 설정
            hess = 1.0  # CatBoost에서는 Hessian을 안 쓰는 경우가 많음
            
            derivatives.append(grad)
            second_derivatives.append(hess)
        
        return zip(derivatives, second_derivatives)

In [134]:
model = CatBoostRegressor(
    iterations=2000,
    depth=7,
    learning_rate=0.05,
    loss_function=MultiObjectiveLoss(alpha=0.7, beta=0.3),  # 가중치 조정 가능
    random_seed=42,
    verbose=200
)

In [135]:
model.fit(X_train, y_train)

CatBoostError: catboost/private/libs/target/data_providers.cpp:639: Currently only multi-regression, multilabel and survival objectives work with multidimensional target

### 방법 3 :  CatBoost의 MultiRegression 모드

In [57]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(740, 6)
(740, 3)
(186, 6)
(186, 3)


In [58]:
model = CatBoostRegressor(
    iterations=2000,
    depth=7,
    learning_rate=0.05,
    loss_function="MultiRMSE",  # MultiRegression을 위한 손실 함수
    random_seed=42,
    verbose=200
)

In [59]:
model.fit(X_train, y_train)


0:	learn: 0.3564912	total: 14.4ms	remaining: 28.9s
200:	learn: 0.1020031	total: 450ms	remaining: 4.03s
400:	learn: 0.0737576	total: 853ms	remaining: 3.4s
600:	learn: 0.0603022	total: 1.25s	remaining: 2.9s
800:	learn: 0.0526923	total: 1.62s	remaining: 2.43s
1000:	learn: 0.0478308	total: 2.01s	remaining: 2.01s
1200:	learn: 0.0446486	total: 2.39s	remaining: 1.59s
1400:	learn: 0.0424420	total: 2.77s	remaining: 1.19s
1600:	learn: 0.0407968	total: 3.17s	remaining: 789ms
1800:	learn: 0.0395850	total: 3.56s	remaining: 393ms
1999:	learn: 0.0386691	total: 3.95s	remaining: 0us


In [60]:
y_preds = model.predict(X_test)
print(y_preds.shape)
print(type(y_preds))

(186, 3)
<class 'numpy.ndarray'>


In [61]:
print(y_test.shape)
print(type(y_test))

(186, 3)
<class 'numpy.ndarray'>


In [62]:
import numpy as np

def catboost_multi_evaluate(model, y_train, y_pred, y_test, target_cols):
    rmse_list, mae_list, r2_list = [], [], []
    
    for idx, col in enumerate(target_cols):
        y_true_i = y_test[:, idx]  # target_dict의 key를 인덱스로 변환
        y_pred_i = y_pred[:, idx]
        
        mae_i = np.mean(np.abs(y_true_i - y_pred_i))
        mse_i = np.mean((y_true_i - y_pred_i) ** 2)
        rmse_i = np.sqrt(mse_i)
        
        sse_i = np.sum((y_true_i - y_pred_i) ** 2)
        sst_i = np.sum(y_true_i - np.mean(y_train) ** 2)
        
        r2_i = 1 - sse_i / sst_i
        
        rmse_list.append(rmse_i)
        mae_list.append(mae_i)
        r2_list.append(r2_i)
        
        print(f"Target '{col}' - RMSE: {rmse_i:.4f}, MAE: {mae_i:.4f}, R2: {r2_i:.4f}")
    
    rmse_mean = np.mean(rmse_list)
    mae_mean = np.mean(mae_list)
    r2_mean = np.mean(r2_list)
    
    print(f"[Average Metrics] RMSE: {rmse_mean:.4f}, MAE: {mae_mean:.4f}, R2: {r2_mean:.4f}")
    return rmse_mean, mae_mean, r2_mean

In [63]:
rmse_mean, mae_mean, r2_mean = catboost_multi_evaluate(model,
                            y_train,    
                            y_preds,  
                            y_test,   
                            target_cols)
print(f'catboost 모델 mulit-objective RMSE: {rmse_mean:.4f}')
print(f'catboost 모델 mulit-objective MAE: {mae_mean:.4f}')
print(f'catboost 모델 mulit-objective R^2: {r2_mean:.4f}')

Target 'strength' - RMSE: 0.0791, MAE: 0.0427, R2: 0.9684
Target 'cement' - RMSE: 0.0701, MAE: 0.0329, R2: 0.9729
Target 'water' - RMSE: 0.0650, MAE: 0.0318, R2: 0.9873
[Average Metrics] RMSE: 0.0714, MAE: 0.0358, R2: 0.9762
catboost 모델 mulit-objective RMSE: 0.0714
catboost 모델 mulit-objective MAE: 0.0358
catboost 모델 mulit-objective R^2: 0.9762


In [64]:
# 모델저장
def catboost_save(model, path):
    """
    CatBoost 모델을 지정된 경로에 저장합니다.

    Args:
        model (CatBoostRegressor): 저장할 CatBoost 모델 객체
        path (str): 모델을 저장할 파일 경로

    Returns:
        None
    """
    model.save_model(path, format='cbm')

In [65]:
catboost_save(model,'./model_save/catboost_multi_model.cbm')

In [66]:
# 모델불러오기
def catboost_load(path):
    """CatBoost 모델 불러오기"""
    model = CatBoostRegressor()  # 회귀 모델이면 CatBoostRegressor, 분류 모델이면 CatBoostClassifier
    model.load_model(path)
    return model

In [69]:
loaded_model = catboost_load('./model_save/catboost_multi_model.cbm')

In [70]:
y_pred = loaded_model.predict(X_test)

## Surrogate Model(TabPFN)으로 multi-objective 구현

In [4]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(740, 8)
(740, 1)
(186, 8)
(186, 1)


In [104]:
from tabpfn import TabPFNRegressor

In [105]:
models = []
for i in range(y_train.shape[1]):
    model = TabPFNRegressor()
    # model.fit(X_train, y_train.iloc[:, i])
    model.fit(X_train, y_train[:, i])
    models.append(model)

In [107]:
y_preds = []
for m in models:
    y_pred = m.predict(X_test)
    print(y_pred.shape)
    if y_pred.ndim == 1:
        y_pred = y_pred.reshape(-1, 1)
    print(y_pred.shape)
    y_preds.append(y_pred)
y_preds = np.column_stack(y_preds)
print(y_preds.shape, type(y_preds))

(186,)
(186, 1)
(186,)
(186, 1)
(186,)
(186, 1)
(186, 3) <class 'numpy.ndarray'>


In [108]:
import numpy as np

def tabpfn_multi_evaluate(model, y_train, y_pred, y_test, target_cols):
    rmse_list, mae_list, r2_list = [], [], []
    
    for idx, col in enumerate(target_cols):
        y_true_i = y_test[:, idx]  # target_dict의 key를 인덱스로 변환
        y_pred_i = y_pred[:, idx]
        
        mae_i = np.mean(np.abs(y_true_i - y_pred_i))
        mse_i = np.mean((y_true_i - y_pred_i) ** 2)
        rmse_i = np.sqrt(mse_i)
        
        sse_i = np.sum((y_true_i - y_pred_i) ** 2)
        sst_i = np.sum(y_true_i - np.mean(y_train) ** 2)
        
        r2_i = 1 - sse_i / sst_i
        
        rmse_list.append(rmse_i)
        mae_list.append(mae_i)
        r2_list.append(r2_i)
        
        print(f"Target '{col}' - RMSE: {rmse_i:.4f}, MAE: {mae_i:.4f}, R2: {r2_i:.4f}")
    
    rmse_mean = np.mean(rmse_list)
    mae_mean = np.mean(mae_list)
    r2_mean = np.mean(r2_list)
    
    print(f"[Average Metrics] RMSE: {rmse_mean:.4f}, MAE: {mae_mean:.4f}, R2: {r2_mean:.4f}")
    return rmse_mean, mae_mean, r2_mean

In [109]:
rmse_mean, mae_mean, r2_mean = tabpfn_multi_evaluate(model,
                            y_train,    
                            y_preds,  
                            y_test,   
                            target_cols)
print(f'tabpfn 모델 mulit-objective RMSE: {rmse_mean:.4f}')
print(f'tabpfn 모델 mulit-objective MAE: {mae_mean:.4f}')
print(f'tabpfn 모델 mulit-objective R^2: {r2_mean:.4f}')

Target 'strength' - RMSE: 0.0878, MAE: 0.0455, R2: 0.9611
Target 'cement' - RMSE: 0.0604, MAE: 0.0140, R2: 0.9799
Target 'water' - RMSE: 0.0476, MAE: 0.0112, R2: 0.9932
[Average Metrics] RMSE: 0.0653, MAE: 0.0236, R2: 0.9781
tabpfn 모델 mulit-objective RMSE: 0.0653
tabpfn 모델 mulit-objective MAE: 0.0236
tabpfn 모델 mulit-objective R^2: 0.9781
